##### Steps for importing data:
- download sra toolkit
- prefetch SRA files for patients P014, P020, P027, P044 and P061 (only patients where pre- and post treatment data was available; also limiting the amount of data was necessary because of processing power limitations)(prefetch SRR23801859 SRR23801959 SRR23801860 SRR23801941 SRR23801866 SRR23801999 SRR23801898 SRR23801944 SRR23801908 SRR23801935 in command shell)
- fastq-dump the same files (fastq-dump --outdir fastq --gzip --skip-technical  --readids --read-filter pass --dumpbase --split-3 --clip in command shell)
- import data into jupyterhub
- save data as qza artifact (manifest file was written manually in notepad)

In [36]:
# importing all required packages & notebook extensions at the start of the notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import qiime2 as q2
import scipy.stats as stats
from qiime2 import Artifact

# This is to supress the warning messages (if any) generated in our code
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [8]:
raw_data_dir = "../data/raw"
data_dir = "../data/processed"
vis_dir  = "../results"

In [27]:
! qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path $raw_data_dir/manifest.csv \
  --input-format PairedEndFastqManifestPhred33V2 \
  --output-path $data_dir/demux-paired-end.qza

Imported ../data/raw/manifest.csv as PairedEndFastqManifestPhred33V2 to ../data/processed/demux-paired-end.qza


In [39]:
! qiime assembly assemble-megahit \
    --i-seqs  /data/w9_data/reads.qza \
    --p-presets meta-sensitive \
    --p-num-cpu-threads 3 \
    --o-contigs $data_dir/contigs.qza


Usage: qiime assembly assemble-megahit [OPTIONS]

  This method uses MEGAHIT to assemble provided paired- or single-end NGS
  reads into contigs.

Inputs:
  --i-seqs ARTIFACT SampleData[SequencesWithQuality |
    PairedEndSequencesWithQuality]
                          The paired- or single-end sequences to be
                          assembled.                                [required]
Parameters:
  --p-presets TEXT Choices('meta', 'meta-sensitive', 'meta-large',
    'disabled')           Override a group of parameters. See the megahit
                          documentation for details.                [optional]
  --p-min-count INTEGER   Minimum multiplicity for filtering (k_min+1)-mers.
    Range(1, None)                                                [default: 2]
  --p-k-list INTEGERS... Range(15, 255, inclusive_end=True)
                          List of kmer size - all must be odd with an
                          increment <= 28.
                                 [default: [21, 